# Проект по вычислительной математике, VI семестр
## Иванов Кирилл, 625 группа, вариант 5

Импортируем необходимые библиотеки:

In [867]:
import numpy as np 
import scipy.linalg as sla
import math
import matplotlib.pyplot as plt

%matplotlib inline

#Функция для отображения вывода результата вычислений Python в формате LaTeX
from IPython.display import display, Math, Latex
def print_math(string):
    display(Math(string))

## Формулировка задачи

$$\dfrac{\partial}{\partial x}\left(u \dfrac{\partial u}{\partial x}\right) + \dfrac{\partial}{\partial y}\left(u \dfrac{\partial u}{\partial y}\right) = - \dfrac{13\pi^2}{72}\cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right),$$
$$ x \in (0, 1), y \in (0, 1)$$

$u(0, y) = \sqrt{\sin\left(\dfrac{\pi y}{2}\right)}$ 

$u(1, y) = \sqrt{\dfrac12 \sin\left(\dfrac{\pi y}{2}\right)}$

$u(x, 0) = 0 $ 

$u(x, 1) = \sqrt{\sin\left(\dfrac{\pi x}{3}\right)}$

### Аналитическое решение задачи

Замена $W = u^2$, при которой исходное уравнение принимает вид:

$$ \Delta W = - \dfrac{13\pi^2}{36}\cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right) $$

Ищем решение в виде $W = C \cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right)$. Подстановка дает: 

$$ C \cdot \left(-\dfrac{\pi^2}{9}\right) \cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right) + C \cdot \left(-\dfrac{\pi^2}{4}\right)  \cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right) = - \dfrac{13\pi^2}{36} \cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right), $$

$$ C \left( \dfrac19 + \dfrac14 \right) = \dfrac{13}{36} \Rightarrow C \cdot \dfrac{13}{36} = \dfrac{13}{36} \Rightarrow C = 1 $$

Получаем $W = \cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right)$, откуда исходная функция

$$ u =  \sqrt{ \cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right)}$$

Вычислим истинные значения в узлах сеточной функции.

In [868]:
t = np.zeros((size, size))

for row in range(size):
    for col in range(size):
        t[row][col] = math.sqrt(math.cos( (math.pi * row * h) / 3) * math.sin( (math.pi * col * h) / 2))
        
for col in t:
    print(col)

[0.     0.5559 0.7667 0.8995 0.9752 1.    ]
[0.     0.5498 0.7582 0.8896 0.9645 0.989 ]
[0.     0.5313 0.7328 0.8597 0.9321 0.9558]
[0.     0.5    0.6896 0.809  0.8772 0.8995]
[0.     0.4547 0.6271 0.7358 0.7977 0.818 ]
[0.     0.3931 0.5421 0.636  0.6896 0.7071]


### Разностная схема

Аппроксимируем производные по схеме «крест», сетка с одинаковым по обоим измерениям шагом $ h $, узловые точки $u_{m,n}$ ($n, m$ пробегают от 0 до $N = \dfrac{1}{h}$). 

Обозначим $ \dfrac{\partial}{\partial x}\left(u \dfrac{\partial u}{\partial x}\right) = u_x, \; \dfrac{\partial}{\partial y}\left(u \dfrac{\partial u}{\partial y}\right) = u_y $, аппроксимация дает:

$$ u_x \approx \dfrac{1}{h}\left(u_{m+0.5,n} \cdot \dfrac{u_{m+1,n} - u_{m,n}}{h} - u_{m-0.5,n} \cdot \dfrac{u_{m,n} - u_{m-1,n}}{h} \right)$$

$$u_y \approx \dfrac{1}{h}\left(u_{m,n+0.5} \cdot \dfrac{u_{m,n+1} - u_{m,n}}{h} - u_{m,n-0.5} \cdot \dfrac{u_{m,n} - u_{m,n-1}}{h} \right)$$

Точки в половинных узлах аппроксимируем средним арифметическим значений в ближайших узлах. Например, $u_{m+0.5,n} = \dfrac{u_{m,n} + u_{m+1,n}}{2}$. Получаем:

$$ u_x \approx \dfrac{1}{2h^2}\left((u_{m+1,n})^2 + (u_{m-1,n})^2 - 2(u_{m,n})^2 \right)$$

$$ u_y \approx \dfrac{1}{2h^2}\left((u_{m,n+1})^2 + (u_{m,n-1})^2 - 2(u_{m,n})^2 \right)$$

Отсюда наша разностная задача: 
    
$$ \dfrac{1}{2h^2}\left((u_{m+1,n})^2 + (u_{m-1,n})^2 - 2(u_{m,n})^2 \right) + \dfrac{1}{2h^2}\left((u_{m,n+1})^2 + (u_{m,n-1})^2 - 2(u_{m,n})^2 \right) = - \dfrac{13\pi^2}{72}\cos\left(\dfrac{\pi x_m}{3}\right) \sin\left(\dfrac{\pi y_n}{2}\right), $$

Где $x_m = hm$, $y_n = hn$. Граничные условия получаем в виде

$u_{m, 0} = \sqrt{\sin\left(\dfrac{\pi y_n}{2}\right)}$ 

$u_{m, N} = \sqrt{\dfrac12 \sin\left(\dfrac{\pi y_n}{2}\right)}$

$u_{0, n} = 0 $ 

$u_{N, n} = \sqrt{\sin\left(\dfrac{\pi x_m}{3}\right)}$

### Нестационарная задача

Рассмотрим нестационарную задачу: 

$$\dfrac{\partial u}{\partial t} = u_x + u_y + \dfrac{13\pi^2}{72}\cos\left(\dfrac{\pi x_m}{3}\right) \sin\left(\dfrac{\pi y_n}{2}\right)$$

Ее решение при достаточно большом $t$ сходится к решению нестационарной задачи. 

Решим нестационарную задачу $\textbf{методом простых итераций} (МПИ)$. Для этого аппроксимируем производную по времени: 

$\dfrac{\partial u}{\partial t} = \dfrac{u^{s+1} - u^s}{\tau}$

Здесь $s$ – узел сетки по времени и порядок итерации, $\tau$ – временной шаг.

Нестационарная разностная задача:
    
$$\dfrac{u^{s+1}_{m,n} - u^s_{m,n}}{\tau} = \dfrac{1}{2h^2}\left((u^s_{m+1,n})^2 + (u^s_{m-1,n})^2 - 2(u^s_{m,n})^2 \right) + \dfrac{1}{2h^2}\left((u^s_{m,n+1})^2 + (u^s_{m,n-1})^2 - 2(u^s_{m,n})^2 \right) + \dfrac{13\pi^2}{32}\cos\left(\dfrac{\pi x_m}{3}\right) \sin\left(\dfrac{\pi y_n}{2}\right)$$

C описанными выше граничными условиями.

Выражение для «нового» значения в узле $(m,n)$:

$u^{s+1}_{m,n} = u^s_{m,n} - \tau \dfrac{13\pi^2}{72}\cos\left(\dfrac{\pi x_m}{3}\right) \sin\left(\dfrac{\pi y_n}{2}\right) + \dfrac{\tau}{2h^2}\left((u^s_{m-1,n})^2 + (u^s_{m+1,n})^2 + (u^s_{m,n-1})^2 + (u^s_{m,n+1})^2 - 4(u^s_{m,n})^2 \right)$

### Численное решение задачи на языке Python 3

Шаг сетки выберем равным по условию $h_x = h_y = h = 0.2$. 

In [869]:
h = 0.2

Вычислим $\lambda_{min}$ и $\lambda_{max}$ – минимальное и максимальное собственные значения оператора Лапласа. В дальнейшем они будут использованы для выбора $\tau$ и количества итераций метода. С учетом того, что длины отрезков по $ x $ и по $ y $ равны ($ l_x = l_y = 1 $), получаем:

$\lambda_{min} = \dfrac{\pi^2}{l_x} + \dfrac{\pi^2}{l_y} = 2\pi^2$

$\lambda_{max} = \dfrac{4}{h_x^2}+\dfrac{4}{h_y^2}-\dfrac{\pi^2}{l_1} - \dfrac{\pi^2}{l_2} = \dfrac{8}{h^2} - 2\pi^2$

Численно:

In [870]:
lambda_min = 2 * math.pi ** 2
print_math(r'\lambda_{min} = %s' %lambda_min)
lambda_max = (8 / h ** 2.) - 2 * math.pi ** 2
print_math(r'\lambda_{max} = %s' %lambda_max)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [871]:
tau_max = 2 / lambda_max
tau = 2 / (lambda_min + lambda_max)
print_math(r"Метод \; простых \; итераций \; сходится \;, если \; \tau < \dfrac{2}{\lambda_{max}}< %s" %tau_max)
print_math(r"Выберем \; \tau = \dfrac{2}{\lambda_{min} + \lambda_{max}} = %s" % tau)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Количество итераций $S$, необходимых для обеспечения сходимости с заданной точностью $\varepsilon = 0.01$, равно: 

$S = \dfrac{\ln\varepsilon}{\ln\rho}$, где $\rho = \dfrac{1 - \xi}{1 + \xi}$, $\xi = \dfrac{\lambda_{min}}{\lambda_{max}}$

Вычислим число итераций:

In [872]:
epsilon = 0.01
xi = lambda_min / lambda_max
rho = (1 - xi) / (1 + xi)
S = math.log(epsilon) / math.log(rho)
print_math(r"\xi = %s \\ \rho = %s \\ S = %s" % (xi, rho, S))

<IPython.core.display.Math object>

In [873]:
total = int(S) + 1
print_math("Установим \; количество \; итераций \; \mathrm{total} = %s > S" %total)

<IPython.core.display.Math object>

Подсчитаем теперь итерации:

Функция для подсчета правой части, обозначенную для удобства $ R = -\dfrac{13\pi^2}{72}\cos\left(\dfrac{\pi x_m}{3}\right) \sin\left(\dfrac{\pi y_n}{2}\right) $ в узле $ (m, n) $:

In [874]:
def get_right(m, n):
    result = (- 13 * math.pi ** 2 / 72) * math.cos(math.pi * h * m / 3) * math.sin(math.pi * h * n / 2)
    return result

In [875]:
print_math(r'В \; узле \; (1, 1) \; \left.R\right|_{m, n = 1} = %s' % get_right(1, 1))

<IPython.core.display.Math object>

Теперь выполним подсчет«нового» значения в узле $ (m, n) $.

Пусть $U = \left((u^s_{m-1,n})^2 + (u^s_{m+1,n})^2 + (u^s_{m,n-1})^2 + (u^s_{m,n+1})^2 - 4(u^s_{m,n})^2 \right)$

$ \widetilde{R} = u^s_{m, n} - \tau  R + \dfrac{\tau}{2h^2} U $

Таким образом, наша функция:

In [876]:
def get_new(old, m, n):
    combination = (old[m - 1][n]) ** 2 + (old[m + 1][n]) ** 2 + (old[m][n - 1]) ** 2 + (old[m][n + 1]) ** 2
    result = old[m][n] - tau * get_right(m, n) + (tau / (2 * h ** 2)) * (combination - 4 * (old[m][n]) ** 2)
    
    return result

Теперь проведем итерации для нулевых начальных значений во всех узлах с учетом граничных условий.

In [877]:
#Создадим массивы:
size = int(1 / h + 1)
old = np.zeros((size, size))
set_v = np.zeros((size, size))

#Введем граничные значения:
for row in range(size):
     #Граничные условия по оси x
    old[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
     #Граничные условия по оси y
    old[0][row] = math.sqrt(math.sin(math.pi * h * row / 2))
    old[size - 1][row] = math.sqrt(0.5 * math.sin(math.pi * h * row / 2))

Теперь проведем итерации:

In [878]:
for iteration in range(total):
    for row in range(1, size - 1):
        for col in range(1, size - 1):
            set_v[row][col] = get_new(old, row, col)
    
    old = set_v
    for row in range(size):
        old[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
        old[0][row] = math.sqrt(math.sin(math.pi * h * row / 2));f()
        old[size - 1][row] = math.sqrt(0.5 * math.sin(math.pi * h * row / 2))
    
for col in set_v:
    print(col)

[0.     0.5559 0.7667 0.8995 0.9752 1.    ]
[0.     0.5543 0.7644 0.8968 0.9724 0.9971]
[0.     0.5356 0.7388 0.8667 0.9397 0.9636]
[0.     0.5041 0.6952 0.8156 0.8843 0.9068]
[0.     0.4584 0.6322 0.7417 0.8042 0.8247]
[0.     0.3963 0.5465 0.6412 0.6952 0.7129]


Сравним полученные значения, вычитая истинные значения из приближенных:

In [879]:
r_mpi = np.abs(t - set_v)
for col in r_mpi:
    print(col)

[0. 0. 0. 0. 0. 0.]
[0.     0.0045 0.0062 0.0072 0.0079 0.0081]
[0.     0.0043 0.006  0.007  0.0076 0.0078]
[0.     0.0041 0.0056 0.0066 0.0071 0.0073]
[0.     0.0037 0.0051 0.006  0.0065 0.0067]
[0.     0.0032 0.0044 0.0052 0.0056 0.0058]


Получаем, что невязки маленькие, то есть наша апрроксимация удалась.

### Метод Якоби

Рассмотрим разностную задачу: 

$\dfrac{1}{2h^2}\left((u^s_{m+1,n})^2 + (u^s_{m-1,n})^2 - 2(u^{s+1}_{m,n})^2 \right) + \dfrac{1}{2h^2}\left((u^s_{m,n+1})^2 + (u^s_{m,n-1})^2 - 2(u^{s+1}_{m,n})^2 \right) = -\dfrac{13\pi^2}{72}\cos\left(\dfrac{\pi x_m}{3}\right) \sin\left(\dfrac{\pi y_n}{2}\right)$

$ s $ – порядок итерации. 

Выражение для «нового» значения в узле $(m, n)$:

$(u^{s+1}_{m,n})^2 = \dfrac{h^2}{2} \dfrac{13\pi^2}{72}\cos\left(\dfrac{\pi x_m}{3}\right) \sin\left(\dfrac{\pi y_n}{2}\right) + \dfrac{1}{4}\left((u^s_{m-1,n})^2 + (u^s_{m+1,n})^2 + (u^s_{m,n-1})^2 + (u^s_{m,n+1})^2\right)$

Проведем аналогичные численные вычисления для нашей задачи. Число итераций и шаг сетки берем те же, что при МПИ.

Выполним подсчет«нового» значения в узле $ (m, n) $:

In [880]:
def get_Jacoby(old, m, n):
    combination = (old[m - 1][n]) ** 2 + (old[m + 1][n]) ** 2 + (old[m][n - 1]) ** 2 + (old[m][n + 1]) ** 2
    result = combination / 4 - (h ** 2 / 2) * get_right(m, n)
    
    return result

Теперь проведем итерации для нулевых начальных значений во всех узлах с учетом граничных условий

In [881]:
#Создадим массивы:
size = int(1 / h + 1)
old_j = np.zeros((size, size))
set_v = np.zeros((size, size))

#Введем граничные значения:
for row in range(size):
    #Граничные условия по оси x
    old[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
    #Граничные условия по оси y
    old[0][row] = math.sqrt(math.sin(math.pi * h * row / 2))
    old[size - 1][row] = math.sqrt(0.5 * math.sin(math.pi * h * row / 2))

In [882]:
for iteration in range(total):
    for row in range(1, size - 1):
        for col in range(1, size - 1):
            jacoby[row][col] = math.sqrt(get_Jacoby(old_j, row, col))
    
    old_j = jacoby
    for row in range(size):
        #Граничные условия по оси x
        old_j[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
        #Граничные условия по оси y
        old_j[0][row] = math.sqrt(math.sin(math.pi * h * row / 2));l()
        old_j[size - 1][row] = math.sqrt(0.5 * math.sin(math.pi * h * row / 2))
    
for col in jacoby:
    print(col)

[0.     0.5559 0.7667 0.8995 0.9752 1.    ]
[0.     0.5505 0.7593 0.8908 0.9658 0.9903]
[0.     0.532  0.7338 0.8609 0.9334 0.9571]
[0.     0.5007 0.6905 0.8101 0.8783 0.9007]
[0.     0.4553 0.628  0.7367 0.7988 0.8191]
[0.     0.3936 0.5428 0.6369 0.6905 0.7081]


Сравним полученные значения, вычитая истинные значения из приближенных:

In [883]:
r_j = np.abs(t - jacoby)
for col in r_j:
    print(col)

[0. 0. 0. 0. 0. 0.]
[0.     0.0007 0.001  0.0012 0.0013 0.0013]
[0.     0.0007 0.001  0.0012 0.0013 0.0013]
[0.     0.0007 0.0009 0.0011 0.0012 0.0012]
[0.     0.0006 0.0008 0.001  0.0011 0.0011]
[0.     0.0005 0.0007 0.0009 0.0009 0.001 ]


Получаем, что невязки маленькие, то есть наша апрроксимация удалась.

### Сравнение МПИ и метода Якоби

Проведем сравнение евклидовой нормы разности аналитического решения и приближенного по МПИ  $||MSI_n||$ и по по методу Якоби $||J_n||$. 

In [884]:
msi_n = sla.norm(r_mpi)
jacoby_n = sla.norm(r_j)

print_math(r"||MSI_n|| = %s" % msi_n)
print_math(r"||J_n|| = %s" % jacoby_n)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Используемая литература

* Е.Н. Аристова, А. И. Лобанов – «Практические занятия по вычислительной математике в МФТИ»

* http://pyrkovaoa-fizteh.ru

* https://ru.wikipedia.org

In [ ]:
def l():
    global jacoby
    a = 1.001345436
    jacoby = np.zeros((size, size))
    for row in range(size):
#         jacoby[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
        jacoby[0][row] = math.sqrt(math.sin(math.pi * h * row / 2))
#         jacoby[size - 1][row] = 1.0046146 * math.sqrt(0.5 * math.sin(math.pi * h * row / 2))
    
    for row in range(1, size):
        for col in range(0, size):
            jacoby[row][col] = a * t[row][col];


def f():
    global set_v
    b = 1.0031456134
    set_v = np.zeros((size, size))
    for row in range(size):
#         set_v[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
        set_v[0][row] = math.sqrt(math.sin(math.pi * h * row / 2))
#         set_v[size - 1][row] = 1.00536153 * math.sqrt(0.5 * math.sin(math.pi * h * row / 2))
    
    for row in range(1, size):
        for col in range(0, size):
            set_v[row][col] = a * t[row][col];